## 非线性多分类 SVM
---

本节讲述如何在 iris 数据集上实现多分类的高斯内核函数。

高斯内核函数：

$$K(x_1, x_2) = \exp(-\gamma * \|x_1 - x_2\|^2)$$

我们选择分析的数据源：

- X: (Sepal Length, Petal Width)
- Y: (I. setosa, I. virginica, I. versicolor) （三个分类）

基本思想：引入一个额外的维度来进行 1 vs 所有分类。

对点的预测将是具有最大边距或者离边界最远的类别。

好了，开码：

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn import datasets
from tensorflow.python.framework import ops
ops.reset_default_graph()

### 启动计算图会话

In [2]:
sess = tf.Session()

### 加载 iris 数据

In [3]:
# Load the data
# iris.data = [(Sepal Length, Sepal Width, Petal Length, Petal Width)]
iris = datasets.load_iris()
x_vals = np.array([[x[0], x[3]] for x in iris.data])
y_vals1 = np.array([1 if y==0 else -1 for y in iris.target])
y_vals2 = np.array([1 if y==1 else -1 for y in iris.target])
y_vals3 = np.array([1 if y==2 else -1 for y in iris.target])
y_vals = np.array([y_vals1, y_vals2, y_vals3])
class1_x = [x[0] for i, x in enumerate(x_vals) if iris.target[i] == 0]
class1_y = [x[1] for i, x in enumerate(x_vals) if iris.target[i] == 0]
class2_x = [x[0] for i, x in enumerate(x_vals) if iris.target[i] == 1]
class2_y = [x[1] for i, x in enumerate(x_vals) if iris.target[i] == 1]
class3_x = [x[0] for i, x in enumerate(x_vals) if iris.target[i] == 2]
class3_y = [x[1] for i, x in enumerate(x_vals) if iris.target[i] == 2]

### 声明占位符并创建模型变量

In [4]:
batch_size = 50

# Initialize placeholders
x_data = tf.placeholder(shape=[None, 2], dtype=tf.float32)
y_target = tf.placeholder(shape=[3, None], dtype=tf.float32)
prediction_grid = tf.placeholder(shape=[None, 2], dtype=tf.float32)

# Create variables for svm
b = tf.Variable(tf.random_normal(shape=[3, batch_size]))

### 创建高斯内核函数

In [5]:
# Gaussian (RBF) kernel
gamma = tf.constant(-10.0)
dist = tf.reshape(tf.reduce_sum(tf.square(x_data), 1), [-1, 1])
sq_dists = tf.multiply(2., tf.matmul(x_data, tf.transpose(x_data)))
my_kernel = tf.exp(tf.multiply(gamma, tf.abs(sq_dists)))

声明一个函数来重塑矩阵和做批量矩阵乘法：

In [ ]:
# Declare function to do reshape/batch multiplication
def reshape_matmul(mat):
    v1 = tf.expand_dims(mat, 1)
    v2 = tf.reshape(v1, [3, batch_size, 1])
    return 